In [7]:
from lib.llm_setup import create_input_message
from peft import PeftConfig
from transformers import AutoTokenizer

lora_adapter = "IlyaGusev/saiga2_13b_lora"
# lora_adapter = "aesedeu/openorca-7b-stlt-chatbot-lora"
config = PeftConfig.from_pretrained(lora_adapter)
base_model = config.base_model_name_or_path

# Настройка токенизатора
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.bos_token = "<s>"
tokenizer.eos_token = "</s>"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

print(
    create_input_message(
    question='Как играть?',
    tokenizer=tokenizer
))

<s>system
Ты - русскоязычный ассистент Степан. Отвечаешь на вопросы людей и помогаешь им.</s>
<s>user
Как играть?</s>
<s>assistant



# тестовые ячейки

In [1]:
import json
f = open("./dialogs/q_a.json")

data = json.load(f)
data

{'1707831919.223918': {'date': '2024-02-13',
  'time_utc': '13:45:19',
  'question': 'привет',
  'response': 'Привет!',
  'time_spent': 0.36},
 '1707832482.158092': {'date': '2024-02-13',
  'time_utc': '13:54:42',
  'question': 'Ты кто?',
  'response': 'Я - языковой ассистент Степан. Моя задача помогать пользователям с вопросами о лотереях, используя только эту информацию:\nПравила лотереи «Забава от Русского лоto» https://www.stoloto.ru/zabava/.\nТакже я могу предоставить дополнительную информацию о лотереях, если она будет нужна для ответа на конкретный вопрос.',
  'time_spent': 6.96},
 '1707832488.40438': {'date': '2024-02-13',
  'time_utc': '13:54:48',
  'question': 'Привет! Я купил билет, но не вижу его в ЛК почему?',
  'response': '\nПривет! Если Вы не видите свой билет в ЛК, возможно, он еще не был обработан системой. Обычно это происходит в течение нескольких минут после покупки билета. Также убедитесь, что Вы правильно ввели данные своего аккаунта при покупке билета. Если проб

## генерация ключей

In [25]:
import secrets

# Генерация случайного ключа
def generate_api_key():
    # Генерация случайной строки из символов шестнадцатеричного формата
    api_key = secrets.token_hex(32)  # Вы можете выбрать любую длину ключа, 32 байта (64 символа шестнадцатеричного формата) в данном случае
    return api_key

# Генерация API ключа
api_key = generate_api_key()
api_key

'716beeee49e8981b54619f960579f798f40dba42e1cf88260a77ddee6ae7b94f'

## POST

In [9]:
import requests
import json

URL = "http://localhost:8008/llm"
POST_JSON = {
    "user_id": 1050,
    "question_string": "Чем ты можешь быть мне полезен?"
}
r = requests.post(
    url=URL,
    headers={
        "Content-Type": "application/json",
        "API": "131e12aa46252f4da6920dd2feccc94978688eab3a96337ba4b67a945eac1308"
    },
    json=POST_JSON
)

response = r.json()
response

{'user_id': 1050,
 'date': '2024-02-13',
 'time': '08:59:14',
 'client_ip': '127.0.0.1',
 'user_question': 'Чем ты можешь быть мне полезен?',
 'ai_response': 'Я могу помочь вам с проверкой результатов лотереи, дать консультации по выбору лотереи и ее номиналов, рассказать о специальных предложениях и акциях. Также я могу помочь с вопросами о процессе регистрации и использования сервиса.',
 'response_time': 4.41}

## Загрузка данных из .csv-файла (book)

In [5]:
from lib.vector_db_setup import vectorstore_query, get_chroma_client

import warnings
warnings.simplefilter("ignore", UserWarning)
import warnings

# Подключаемся к векторному хранилищу
chroma_client = get_chroma_client()
# Получаем коллекцию
collection = chroma_client.get_collection('book')

# question = 'правила лотереи'
question = 'когда тираж забавы'

vector_db_response = vectorstore_query(
    collection=collection,
    source_file_type='table',
    question = question,
    n_results=5
)

print(f'============ QUESTION: ============\n{question}')
print(f'============ VECTOR DB RESPONSE: ============\n{vector_db_response}')

Подключение к CHROMADB: SUCCESS
Доступны следующие коллекции:
- stoloto
- book-txt
- book
============ QUESTION: ============
когда тираж забавы
============ VECTOR DB RESPONSE: ============
Тиражи быстрой лотереи Забава от Русского лото проходят каждые 15 минут часа Правила лотереи «Забава от Русского лото» https://www.stoloto.ru/zabava/rules?int=faq


## Загрузка данных из .txt-файла (book-txt)

In [13]:
from lib.vector_db_setup import vectorstore_query, get_chroma_client

# Подключаемся к векторному хранилищу
chroma_client = get_chroma_client()
# Получаем коллекцию
collection = chroma_client.get_collection('book-txt')

question = 'правила лотереи'
vector_db_response = vectorstore_query(
    collection=collection,
    source_file_type='text',
    question = question,
    n_results=5
)

print(f'============ QUESTION: ============\n{question}')
print(f'============ VECTOR DB RESPONSE: ============\n{vector_db_response}')

Подключение к CHROMADB: SUCCESS
Доступны следующие коллекции:
- stoloto
- book-txt
- book
============ QUESTION: ============
правила лотереи
============ VECTOR DB RESPONSE: ============
Тиражи быстрой лотереи Забава от Русского лото проходят каждые 15 минут часа Возможно, Вы исчерпали ежемесячные лимиты по Кошельку. Вам необходимо пройти идентификацию на сайте Столото или ожидать первого числа нового месяца, когда лимиты обновятся. Чтобы изменить статус подписки, кликните на ее название «(Подписка 1, Подписка 2 и т.д.)» и нажмите «Возобновить/Приостановить», «Удалить». Правила лотереи «Забава от Русского лото» https://www.stoloto.ru/zabava/rules?int=faq
Тиражная комиссия следит за ходом каждого розыгрыша и подтверждает его результаты — подписывает протокол тиражной комиссии и официальную таблицу результатов проведения тиража. Чтобы настроить или отключить рассылки, откройте «Профиль» в личном кабинете и зайдите в раздел «Уведомления».Также вы можете написать по адресу info@stoloto.ru

## Загрузка данных из .csv-файла (stoloto)

In [8]:
from lib.vector_db_setup import vectorstore_query, get_chroma_client
import warnings
warnings.simplefilter("ignore", UserWarning)
import warnings

# Подключаемся к векторному хранилищу
chroma_client = get_chroma_client()
# Получаем коллекцию
collection = chroma_client.get_collection('stoloto')

question = 'как узнать номер тиража'
# question = 'когда тираж забавы'

vector_db_response = vectorstore_query(
    collection=collection,
    source_file_type='table',
    question = question,
    n_results=3
)

print(f'============ QUESTION: ============\n{question}')
print(f'============ VECTOR DB RESPONSE: ============\n{vector_db_response}')

Подключение к CHROMADB: SUCCESS
Доступны следующие коллекции:
- stoloto
- book-txt
- book
============ QUESTION: ============
как узнать номер тиража
============ VECTOR DB RESPONSE: ============
 В настоящий момент может наблюдаться сложность при проверке билета в Мобильном приложении, отсканировав QR-код. Технические специалисты «Столото» оповещены о возникшей сложности и работают над её устранением. Вы можете проверить билет на сайте по ссылке http://www.stoloto/check-ticket . Из предложенного списка необходимо выбрать интересующую Вас лотерею, ввести номер тиража и номер билета, а далее нажать на кнопку Проверить. Если при покупке билета Вы указывали номер телефона, то номер тиража и билета Вы можете посмотреть на черно-белой квитанции, в верхней ее части, после слов «Уведомление о приеме лотерейной ставки и регистрации электронного лотерейного билета». Если Вы не указывали номер телефона, то номер тиража и билета указаны на самом билете. Обратите, пожалуйста, внимание, что многоти

# LLM

In [1]:
from peft import PeftConfig
from transformers import AutoTokenizer
from lib.llm_setup import initialize_lora_model, generate_llm_response

# lora_adapter = "IlyaGusev/saiga_mistral_7b"
lora_adapter = "IlyaGusev/saiga2_13b_lora"
# lora_adapter = "aesedeu/openorca-7b-stlt-chatbot-lora"
config = PeftConfig.from_pretrained(lora_adapter)
base_model = config.base_model_name_or_path

# Настройка токенизатора
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.bos_token = "<s>"
tokenizer.eos_token = "</s>"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

model = initialize_lora_model(
    base_model=base_model,
    lora_adapter=lora_adapter,
    # bnb=True
)

/root/ml_projects/chatbot/9_3_rag_streamlit/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [05:14<00:00, 104.97s/it]


## RAG

In [9]:
from lib.vector_db_setup import get_texts, upload_to_vectorstore, vectorstore_query, get_chroma_client

# Подключаемся к векторному хранилищу
chroma_client = get_chroma_client()
# Получаем коллекцию
collection = chroma_client.get_collection('book-txt')

Подключение к CHROMADB: SUCCESS
Доступны следующие коллекции:
- stoloto
- book
- book-txt


In [12]:
# watch -n0.2 nvidia-smi
question = 'Как потратить бонусы?'
# question = 'Когда следующий тираж забавы?'
# question = "Сколько стоит лотерея забава"

output_message = generate_llm_response(
    question=question,
    model=model,
    collection=collection,
    tokenizer=tokenizer,
    source_file_type='text',
    n_results=3
)

print(output_message)

Бонусы можно использовать для покупки билетов на любые лотереи, которые предлагает СтоЛото. Для этого нужно перейти на страницу с выбором лотерей и выбрать желаемый вид игры. Затем ввести количество билетов и нажать кнопку "Купить". После этого будет отображен размер оплаты, включающий стоимость билетов и сумму бонусов. Если у вас недостаточно бонусов, то придется внести дополнительную сумму.


## no-RAG

In [3]:
# watch -n0.2 nvidia-smi
question = 'Как потратить бонусы?'
# question = 'Когда следующий тираж забавы?'

output_message = generate_llm_response(
    question=question,
    model=model,
    tokenizer=tokenizer
)
print(output_message)

В настоящий момент может наблюдаться сложность при проверке билета в Мобильном приложении, отсканировав QR-код. Технические специалисты «Каспий» оповещены о возникшей сложности и работают над её устранением. Вы можете проверить билет на сайте по ссылке http://www.kaspiy/check-ticket . Из предложенного списка необходимо выбрать интересующую Вас категорию билетов, ввести номер тиража и номер билета, а далее нажать на кнопку Проверить. Если при покупке билета Вы указывали номер телефона, то номер тиража и билета Вы можете посмотреть на черно-белой квитанции, в верхней ее части, после слов „Уведомление о приеме билета и регистрации ти


# langchain
**работает только если в БД искомые ответы лежат в атрибуте "documents", то БД наполнялась НЕ ИЗ ВОПРОСОВ-ОТВЕТОВ**

Модель загружается чуть выше

In [3]:
from chromadb.config import Settings
from chromadb.utils import embedding_functions
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
import numpy as np
import os 
import chromadb
import torch
from peft import PeftConfig
from transformers import AutoTokenizer, GenerationConfig
from transformers import AutoTokenizer
from lib.llm_setup import initialize_model
from transformers import GenerationConfig
from langchain import PromptTemplate

generation_config = GenerationConfig(
  bos_token_id = 1,
  do_sample = True,
  eos_token_id = 2,
  max_length = 2048,
  max_new_tokens = 256,
  repetition_penalty=1.1,
  no_repeat_ngram_size=15,
  pad_token_id = 0,
  temperature = 0.5,
  top_p = 0.9
)

template = "Ты - русскоязычный ассистент Степан. Ты отвечаешь на вопрос пользователя, используя только эту информацию: {context}\nВопрос:{question}\nОтвет:"
prompt = PromptTemplate.from_template(template)

embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

chroma_client = chromadb.HttpClient(settings=Settings(
    allow_reset=True,
    chroma_api_impl='chromadb.api.fastapi.FastAPI',
    chroma_server_host='localhost',
    chroma_server_http_port='8000')
)
collection = chroma_client.get_collection(name="book-txt", embedding_function=embedding_function)

db = Chroma(client=chroma_client,
            collection_name='book-txt',
            embedding_function=embeddings)

retriever = db.as_retriever(
    search_kwargs={"k":3}
)

# question = 'когда следующий тираж забавы'
# db.similarity_search_with_score(question)

/root/ml_projects/chatbot/9_3_rag_streamlit/.venv/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
pipe = pipeline(
        'text-generation',
        model = model,
        tokenizer = tokenizer,
        generation_config=generation_config
)

llm = HuggingFacePipeline(
    pipeline=pipe
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [6]:
# question = 'когда следующий тираж забавы'
question = 'Как потратить бонусы?'
generated_text = qa.invoke({"query": question})

print(f"============QUESTION:============\n{generated_text['query']}\n")
print(f"============RESPONSE:============\n{generated_text['result']}")

============QUESTION:============
Как потратить бонусы?

============RESPONSE:============
Бонусы можно использовать для покупки билетов на любое событие, указанное на сайте stoloto.ru. Бонусы не могут быть использованы для оплаты других услуг или товаров.
Например, если у вас есть 1000 бонусов, то вы можете купить билет на лотерею за 1000 бонусов. Если же стоит билет на лотерею 1000 рублей, то вы можете купить его за 1000 бонусов.


In [7]:
generated_text

{'query': 'Как потратить бонусы?',
 'result': 'Бонусы можно использовать для покупки билетов на любое событие, указанное на сайте stoloto.ru. Бонусы не могут быть использованы для оплаты других услуг или товаров.\nНапример, если у вас есть 1000 бонусов, то вы можете купить билет на лотерею за 1000 бонусов. Если же стоит билет на лотерею 1000 рублей, то вы можете купить его за 1000 бонусов.',
 'source_documents': [Document(page_content='Тиражи быстрой лотереи Забава от Русского лото проходят каждые 15 минут часа', metadata={'source': './SOURCE_DOCUMENTS/answers_stoloto.txt'}),
  Document(page_content='Чтобы изменить статус подписки, кликните на ее название «(Подписка 1, Подписка 2 и т.д.)» и нажмите «Возобновить/Приостановить», «Удалить».', metadata={'source': './SOURCE_DOCUMENTS/answers_stoloto.txt'}),
  Document(page_content='Подробная информация о том, как получить выигрыш, доступна на странице «Как получить выигрыш».\nВыберите способ покупки билетов на странице «Адреса точек продаж 